Basic Data Abstractions

Ordered Facts

In [2]:
# Ordered Facts or Implied Facts represent information as a list of elements

import clips

env = clips.Environment()

# Ordered facts can only be asserted as strings
fact = env.assert_string('(ordered-fact 1 2 3)')

print(fact[0])
print(list(fact))

1
[1, 2, 3]


Template Facts

In [11]:
# Template Facts represent data similarly to Python dictionaries. 
# Require a template to be defined. 
# Templates are formal descriptions of the data represented by the fact.

import clips

template_string = """
(deftemplate person
    (slot name (type STRING))
    (slot surname (type STRING))
    (slot birthdate (type SYMBOL)))
"""

env = clips.Environment()

env.build(template_string)

template = env.find_template('person')

print(f"Template: {template}")

Template: (deftemplate MAIN::person (slot name (type STRING)) (slot surname (type STRING)) (slot birthdate (type SYMBOL)))


In [12]:
fact = template.assert_fact(name='John',
                            surname='Doe',
                            birthdate=clips.Symbol('01/01/1970'))

print(fact)
print(dict(fact))

(person (name "John") (surname "Doe") (birthdate 01/01/1970))
{'name': 'John', 'surname': 'Doe', 'birthdate': '01/01/1970'}


In [13]:
fact.modify_slots(name='Leeroy',
                  surname='Jenkins',
                  birthdate=clips.Symbol('11/05/2005'))

for fact in env.facts():
    print(fact)

(person (name "Leeroy") (surname "Jenkins") (birthdate 11/05/2005))


In [18]:
import clips

env = clips.Environment()

class_string = """
(defclass MyClass (is-a USER)
  (slot One)
  (slot Two))
"""
handler_string = """
(defmessage-handler MyClass handler ()
  (+ ?self:One ?self:Two))
"""
env.build(class_string)
env.build(handler_string)

defclass = env.find_class('MyClass')
instance = defclass.make_instance('instance-name', One=1, Two=2)
retval = instance.send('handler')

assert retval == 3

for instance in env.instances():
    print(instance)

[instance-name] of MyClass (One 1) (Two 2)


Evaluating CLIPS code: quickly evaluate CLIPS statements retrieving their results in Python.

In [19]:
# Create a multifield value.

import clips

env = clips.Environment()

env.eval("(create$ hammer drill saw screw pliers wrench)")

('hammer', 'drill', 'saw', 'screw', 'pliers', 'wrench')

In [22]:
# CLIPS functions can also be called directly without the need of building language specific strings.

import clips

env = clips.Environment()

env.call('create$', clips.Symbol('hammer'), 'drill', 1, 2.0)

('hammer', 'drill', 1, 2.0)

Defining CLIPS Constructs: must be done in CLIPS language

In [25]:
# Rule definition example
# Use the load or the build functions to define the constructs within the engine.

import clips

env = clips.Environment()

rule = """
(defrule my-rule
  (my-fact first-slot)
  =>
  (printout t "My Rule fired!" crlf))
"""
env.build(rule)

for rule in env.rules():
    print(rule)

(defrule MAIN::my-rule (my-fact first-slot) => (printout t "My Rule fired!" crlf))


Embedding Python Code

In [26]:
# Through the define_function method, it is possible to embed Python code within the CLIPS environment.
# The Python function will be accessible within CLIPS via its name as if it was defined via the deffunction construct.
# In this example, Python regular expression support is added within the CLIPS engine.

import re
import clips

def regex_match(pattern, string):
    """Match pattern against string returning a multifield
    with the first element containing the full match
    followed by all captured groups.

    """
    match = re.match(pattern, string)
    if match is not None:
        return (match.group(),) + match.groups()
    else:
        return []

env = clips.Environment()
env.define_function(regex_match)

env.eval('(regex_match "(www.)(.*)(.com)" "www.example.com")')

('www.example.com', 'www.', 'example', '.com')

Facts may be added to the fact-list (using the assert command), removed from the fact-list (using the retract command), modified (using the modify command), or duplicated (using the duplicate command) 

If a fact is asserted into the fact-list that exactly matches an already existing fact, the new assertion will be ignored
(however, this behavior can be changed using the set-fact-duplication function).

Some commands, such as the retract, modify, and duplicate commands, require a fact to be
specified. A fact can be specified either by fact-index or fact-address. Whenever a fact is asserted
it is given a unique integer index called a fact-index. Fact-indices start at one and are incremented
by one for each new fact. When a fact is modified, its fact-index remains unchanged. Whenever a
reset or clear command is given, the fact-indices restart at one. A fact may also be specified
through the use of a fact-address. A fact-address can be obtained by capturing the return value of
commands which return fact addresses (such as assert, modify, and duplicate) or by binding a
variable to the fact address of a fact which matches a pattern on the LHS of a rule

Initial Facts

The deffacts construct allows a set of a priori or initial knowledge to be specified as a collection
of facts. When the CLIPS environment is reset (using the reset command) every fact specified
within a deffacts construct in the CLIPS knowledge base is added to the fact-list.

Initial Objects

The definstances construct allows a set of a priori or initial knowledge to be specified as a
collection of instances of user-defined classes. When the CLIPS environment is reset (using the
reset command) every instance specified within a definstances construct in the CLIPS knowledge
base is added to the instance-list.

Global Variables

The defglobal construct allows variables to be defined which are global in scope throughout the
CLIPS environment.

If
the activations item is being watched (as a result of the watch command), then an
informational message will be displayed each time a rule is activated or deactivated.

Conflict Resolution Strategies: 

depth, breadth, simplicity, complexity, lex, mea, and random. 

The default strategy is depth. 

The current strategy can be set by using the set-strategy command

The preferred mechanisms in CLIPS for ordering the
execution of rules are salience and modules. Salience allows one to explicitly specify that one rule
should be executed before another rule. Modules allow one to explicitly specify that all of the rules
in a particular group (module) should be executed before all of the rules in a different group.